In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glaft.georaster import Raster

In [3]:
import rasterio
# import matplotlib.pyplot as plt
# from rasterio.plot import show as rio_show
# import matplotlib as mpl
import pandas as pd
import geopandas as gpd
import numpy as np

In [4]:
df = pd.read_csv('../results_2022.csv', dtype=str)
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,Vy,SAV-uncertainty-x,SAV-uncertainty-y,SAV-peak-x,SAV-peak-y,SAV-outlier-percent,LSR-uncertainty-nm,LSR-uncertainty-sh
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.32637951498361006,0.35901746648197114,-0.07337931716612034,0.17662068283387966,6.816027574321414,0.0017456245187072372,0.0013862312354439828
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.2904061391872022,0.299206325223178,-0.05369981396402412,0.16505018603597588,2.8578612135434125,0.009696277915313002,0.009108624708324334
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.4480050686250796,0.47890196990956785,-0.031426549357755906,0.1095515493577558,14.40274841437632,0.012744702296493292,0.012744702296493292
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.19932657031566106,0.21744716761708482,-0.0685401991004746,0.20562059730142374,13.832589617204242,0.0075318103275894,0.0075318103275894
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.170416834102238,0.18288635854874324,-0.05834349185116494,0.16871952444650518,6.481863477164512,0.05414784514500587,0.05414784514500587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.17952278944131322,0.2593106958596746,0.04335100854352797,-0.05744697809470645,21.10048596958771,0.007606484754625452,0.007606484754625452
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.11496105665321626,0.15063862595938682,0.041464175857468366,-0.016464174553616817,18.1238530683338,0.006968377551435475,0.006546051639227265
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07502428079962462,0.1110359355834444,0.0405009727221011,-0.021502913882219477,25.219156194671154,0.0032284029549105872,0.0033360163867409403
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07284891177652728,0.11073034590032146,0.04041395796117722,-0.015413956657325617,18.682405636168525,0.004378758498193732,0.004237508224058451


In [5]:
tmp = Raster(df.loc[0, 'Vx'])
a, b = tmp.value_at_coords(621306.41954208, 6738829.50233354, window=3, return_window=True)
vx_grid = rasterio.open(df.loc[0, 'Vx'])
sample_gen_vx = vx_grid.sample([(621306.41954208, 6738829.50233354)])
vx_sampled = np.array([float(record) for record in sample_gen_vx])

In [6]:
# sampled

In [7]:
gps_files = ['/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-04-05_to_2018-03-04_GPS', 
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-08-18_to_2018-08-02_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-03-14_to_2018-03-04_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-06-27_to_2018-05-08_GPS']

# '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_v2_2018-05-08_to_2018-06-27_GPS.csv'

datestrs = ['LS8-20180304-20180405', 'LS8-20180802-20180818', 'Sen2-20180304-20180314', 'Sen2-20180508-20180627']
datenums = [32, 16, 10, 50]

In [8]:
for gps_file, datestr, datenum in zip(gps_files, datestrs, datenums):
    gps = pd.read_csv(gps_file) 
    # Additional treatment for Sen2-20180508-20180627
    # many of the points here should be (nan, nan), (nan, nan) but nans does not work with rio.sample
    if datestr == 'Sen2-20180508-20180627':
        gps.loc[1, 'end_easting'] = 610481.2868266493
        gps.loc[1, 'end_northing'] = 6737102.953712379
        gps.loc[2, 'end_easting'] = 601790.4387747
        gps.loc[2, 'end_northing'] = 6733753.77267354
        gps = gps.loc[0:2]
    gps = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps['end_easting'], gps['end_northing']), crs='EPSG:32607')
    # This is beginning coordinates
    gps_xy = list(gps[['end_easting', 'end_northing']].to_records(index=False))
    # if datestr == 'Sen2-20180508-20180627':
    #     gps_xy[-2] = (601790.43877479, 6733753.77267354)  # it should be (nan, nan), (nan, nan) but nans does not work with rio.sample
    #     gps_xy = gps_xy[:-1]
    print(datestr, gps_xy)
    
    gps['vx (m/d)']  = (gps['start_easting'] - gps['end_easting']) / datenum
    gps['vy (m/d)']  = (gps['start_northing'] - gps['end_northing']) / datenum
    
    df_s = df.loc[df['Date'] == datestr]
    for idx, row in df_s.iterrows():
        vx_grid = Raster(row.Vx)
        vy_grid = Raster(row.Vy)
        sampled = []
        for x, y in gps_xy:
            vx_avg, vx_3by3 = vx_grid.value_at_coords(x, y, window=3, return_window=True)
            vy_avg, vy_3by3 = vy_grid.value_at_coords(x, y, window=3, return_window=True)
            vx_3by3[vx_3by3 < -9998] = np.nan
            vy_3by3[vy_3by3 < -9998] = np.nan
            vx_3by3[vx_3by3 == 0.0] = np.nan    #Vmap
            vy_3by3[vy_3by3 == 0.0] = np.nan    #Vmap
            vx_nn = vx_3by3[0, 1, 1]     # nearest neighbor value
            vy_nn = vy_3by3[0, 1, 1]
            if np.any(~np.isnan(vx_3by3)):
                vx_avg = np.nanmean(vx_3by3)
            else:
                vx_avg = np.nan
            if np.any(~np.isnan(vy_3by3)):
                vy_avg = np.nanmean(vy_3by3)
            else:
                vy_avg = np.nan
                
            vx_nn -= float(df.loc[idx, 'SAV-peak-x'])
            vx_avg -= float(df.loc[idx, 'SAV-peak-x'])
            vy_nn -= float(df.loc[idx, 'SAV-peak-y'])
            vy_avg -= float(df.loc[idx, 'SAV-peak-y'])
            
            sampled.append([vx_nn, vx_avg, vy_nn, vy_avg])
        
        sampled = np.array(sampled)
        # print(row.Vx, float(df.loc[idx, 'SAV-peak-x']), float(df.loc[idx, 'SAV-peak-y']), sampled)
        
        df.loc[idx, 'pt0_vxdiff'] = sampled[0, 0] - gps.loc[0, 'vx (m/d)']
        df.loc[idx, 'pt0_vxavgdiff'] = sampled[0, 1] - gps.loc[0, 'vx (m/d)']
        df.loc[idx, 'pt0_vydiff'] = sampled[0, 2] - gps.loc[0, 'vy (m/d)']
        df.loc[idx, 'pt0_vyavgdiff'] = sampled[0, 3] - gps.loc[0, 'vy (m/d)']
        df.loc[idx, 'pt1_vxdiff'] = sampled[1, 0] - gps.loc[1, 'vx (m/d)']
        df.loc[idx, 'pt1_vxavgdiff'] = sampled[1, 1] - gps.loc[1, 'vx (m/d)']
        df.loc[idx, 'pt1_vydiff'] = sampled[1, 2] - gps.loc[1, 'vy (m/d)']
        df.loc[idx, 'pt1_vyavgdiff'] = sampled[1, 3] - gps.loc[1, 'vy (m/d)']
        df.loc[idx, 'pt2_vxdiff'] = sampled[2, 0] - gps.loc[2, 'vx (m/d)']
        df.loc[idx, 'pt2_vxavgdiff'] = sampled[2, 1] - gps.loc[2, 'vx (m/d)']
        df.loc[idx, 'pt2_vydiff'] = sampled[2, 2] - gps.loc[2, 'vy (m/d)']
        df.loc[idx, 'pt2_vyavgdiff'] = sampled[2, 3] - gps.loc[2, 'vy (m/d)']
        
        # vx_grid = rasterio.open(row.Vx)
        # vy_grid = rasterio.open(row.Vy)
        # sample_gen_vx = vx_grid.sample(gps_xy)
        # vx_sampled = np.array([float(record) for record in sample_gen_vx])
        # sample_gen_vy = vy_grid.sample(gps_xy)
        # vy_sampled = np.array([float(record) for record in sample_gen_vy])
        # vx_sampled[vx_sampled < -9998] = np.nan
        # vy_sampled[vy_sampled < -9998] = np.nan
        # vx_sampled -= float(df.loc[idx, 'SAV-peak-x'])
        # vy_sampled -= float(df.loc[idx, 'SAV-peak-y'])
        # df.loc[idx, 'pt0_vxdiff'] = vx_sampled[0] - np.abs(gps.loc[0, 'vx (m/d)'])
        # df.loc[idx, 'pt0_vydiff'] = vy_sampled[0] - np.abs(gps.loc[0, 'vy (m/d)'])
        # df.loc[idx, 'pt1_vxdiff'] = vx_sampled[1] - np.abs(gps.loc[1, 'vx (m/d)'])
        # df.loc[idx, 'pt1_vydiff'] = vy_sampled[1] - np.abs(gps.loc[1, 'vy (m/d)'])        
        # df.loc[idx, 'pt2_vxdiff'] = vx_sampled[2] - np.abs(gps.loc[2, 'vx (m/d)'])
        # df.loc[idx, 'pt2_vydiff'] = vy_sampled[2] - np.abs(gps.loc[2, 'vy (m/d)'])        
        # v_sampled = np.sqrt(vx_sampled ** 2 + vy_sampled ** 2)
        # df.loc[idx, 'pt0_velodiff'] = v_sampled[0] - np.abs(gps.loc[0, 'velocity (m/d)'])
        # df.loc[idx, 'pt1_velodiff'] = v_sampled[1] - np.abs(gps.loc[1, 'velocity (m/d)'])
        # df.loc[idx, 'pt2_velodiff'] = v_sampled[2] - np.abs(gps.loc[2, 'velocity (m/d)'])

LS8-20180304-20180405 [(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
LS8-20180802-20180818 [(621363.01607688, 6738895.12164604), (610506.52739125, 6737089.56006354), (601790.43877479, 6733753.77267354)]
Sen2-20180304-20180314 [(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
Sen2-20180508-20180627 [(621324.96198502, 6738852.60218059), (610481.28682665, 6737102.95371238), (601790.4387747, 6733753.77267354)]


In [9]:
gps

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),geometry,vx (m/d),vy (m/d)
0,0,2018-06-27,2018-05-08,621348.115449,6.738880e+06,621324.961985,6.738853e+06,36.045525,0.720837,POINT (621324.962 6738852.602),0.463069,0.55252
1,1,2018-06-27,2018-05-08,610481.286827,6.737103e+06,610481.286827,6.737103e+06,NaN,NaN,POINT (610481.287 6737102.954),0.000000,0.00000
2,2,2018-06-27,2018-05-08,NaN,NaN,601790.438775,6.733754e+06,NaN,NaN,POINT (601790.439 6733753.773),NaN,NaN


In [10]:
# gps.loc[0, 'start_easting'] + 1
gps
# gps['end_easting'] - gps['start_easting']

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),geometry,vx (m/d),vy (m/d)
0,0,2018-06-27,2018-05-08,621348.115449,6.738880e+06,621324.961985,6.738853e+06,36.045525,0.720837,POINT (621324.962 6738852.602),0.463069,0.55252
1,1,2018-06-27,2018-05-08,610481.286827,6.737103e+06,610481.286827,6.737103e+06,NaN,NaN,POINT (610481.287 6737102.954),0.000000,0.00000
2,2,2018-06-27,2018-05-08,NaN,NaN,601790.438775,6.733754e+06,NaN,NaN,POINT (601790.439 6733753.773),NaN,NaN


In [11]:
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,...,pt0_vydiff,pt0_vyavgdiff,pt1_vxdiff,pt1_vxavgdiff,pt1_vydiff,pt1_vyavgdiff,pt2_vxdiff,pt2_vxavgdiff,pt2_vydiff,pt2_vyavgdiff
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.095937,0.087257,0.005895,0.013707,0.082225,0.082225,0.512210,0.546933,0.805385,0.777608
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.005945,-0.007944,0.017465,0.018333,0.062545,0.066018,NaN,1.305031,NaN,3.134664
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,-0.118244,-0.116508,-0.153245,-0.154982,0.149294,0.142350,3.657758,3.546646,-3.252546,-3.582407
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,0.004437,0.021798,0.001056,0.001056,0.053225,0.053225,0.569871,0.549038,0.713885,0.675691
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,...,-0.052412,-0.052412,0.022109,0.018637,0.058876,0.058876,5.653424,0.731549,-5.436714,-5.421089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,0.029927,0.024371,0.644149,0.646927,-0.205053,-0.198109,NaN,NaN,NaN,NaN
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,-0.048556,-0.040223,0.521036,0.550202,-0.221036,-0.236314,NaN,NaN,NaN,NaN
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,-0.043517,-0.031017,0.546999,0.556721,-0.228497,-0.242386,NaN,NaN,NaN,NaN
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,...,-0.037106,-0.034329,0.534586,0.540142,-0.259586,-0.266530,NaN,NaN,NaN,NaN


In [12]:
df.to_csv('../results_2022.csv', index=False)